In [1]:
# v2: 3-class labels + expanded features + backtest

# 1) Import v2 pipeline (robust to different working dirs)
try:
    from multi_token_modeling_v2 import run_per_token_pipeline, summarize_results
except Exception:
    import sys, os
    sys.path.append("simplest_ml/w1")
    from multi_token_modeling_v2 import run_per_token_pipeline, summarize_results

# 2) Detect data directory
import os
data_dir = "data" if os.path.isdir("data") else ("simplest_ml/w1/data" if os.path.isdir("simplest_ml/w1/data") else "data")
print(f"Using data_dir = {data_dir}")

# 3) Configure tokens and params
tokens = ["BTC","BNB","SOL","ETH","AVAX","TRON","ARBtrium","Sui","Polygon"]  # start with ["BTC","ETH"] to test quickly
interval, start, end = "1h", "20231030", "20251030"

# Feature options
feature_kwargs = dict(
    use_candle=True,
    use_volatility=True,
    use_momentum=True,
    use_volume=True,
    use_time=True,
    zscore_windows=(48,),   # leakage-safe rolling z-score window (48h)
)

# Label: 3-class with horizon & epsilon noise band
label_mode = "three_class"
label_h    = 4          # 4-hour horizon
label_eps  = 0.001      # 0.1% flat band

# Backtest thresholds and fees
t_long   = 0.60         # long if P(up) >= 0.60
t_short  = 0.60         # short if P(down) >= 0.60
fee_bps  = 5.0          # 5 bps per position change
min_hold = 1            # min holding bars after change

# 4) Run v2 pipeline
results = run_per_token_pipeline(
    tokens=tokens,
    data_dir=data_dir,
    interval=interval,
    start=start,
    end=end,
    windows=(3,6,12,24),
    n_splits=5,
    C=0.5,
    max_iter=2000,
    show_eda=False,
    verbose=True,
    label_mode=label_mode,
    label_h=label_h,
    label_eps=label_eps,
    feature_kwargs=feature_kwargs,
    do_backtest=True,
    t_long=t_long,
    t_short=t_short,
    fee_bps=fee_bps,
    min_hold=min_hold,
)

# 5) Summarize across tokens (includes CAGR/Sharpe/HitRate)
summarize_results(results)


Using data_dir = data

[BTC] ==== PIPELINE START ====
[BTC] Step 1/4 LOAD: rows=17568, cols=11, span=2023-10-30 00:59:59.999000+00:00 → 2025-10-30 23:59:59.999000+00:00, cols_req_present=True
[BTC] Step 3/5 FEATURES+LABEL: windows=(3, 6, 12, 24) mode=three_class H=4 eps=0.001
[BTC] prepare_Xy: input rows=17568 [2023-10-30 00:59:59.999000+00:00 → 2025-10-30 23:59:59.999000+00:00]
[BTC] add_features: start, rows=17568, windows=(3, 6, 12, 24)
[BTC] add_features: generated 67 features; total NaNs after lag=787
[BTC] make_label(3-class): H=4 eps=0.001 counts={1: 7852, -1: 7292, 0: 2424}
[BTC] prepare_Xy: dropped NaNs -> rows 17568 → 17518; features=67
[BTC] Step 4/5 TRAIN: TSCV n_splits=5, C=0.5, max_iter=2000
[BTC] train: fold 1/5 | train=2923 test=2919
[BTC] train: fold 1 metrics | AUC=0.5464 ACC=0.4460
[BTC] train: fold 2/5 | train=5842 test=2919
[BTC] train: fold 2 metrics | AUC=0.5460 ACC=0.4669
[BTC] train: fold 3/5 | train=8761 test=2919
[BTC] train: fold 3 metrics | AUC=0.5726 ACC=0

In [1]:
# v3: OOF-based backtest + threshold tuning + fee tiers

try:
    from multi_token_modeling_v3 import run_per_token_pipeline, summarize_results
except Exception:
    import sys, os
    sys.path.append("simplest_ml/w1")
    from multi_token_modeling_v3 import run_per_token_pipeline, summarize_results

import os
data_dir = "data" if os.path.isdir("data") else ("simplest_ml/w1/data" if os.path.isdir("simplest_ml/w1/data") else "data")
print(f"Using data_dir = {data_dir}")

tokens = ["BTC","BNB","SOL","ETH","AVAX","TRON","ARBtrium","Sui","Polygon"]
feature_kwargs = dict(use_candle=True, use_volatility=True, use_momentum=True, use_volume=True, use_time=True, zscore_windows=(48,))

results = run_per_token_pipeline(
    tokens=tokens,
    data_dir=data_dir,
    interval="1h",
    start="20231030",
    end="20251030",
    windows=(3,6,12,24),
    n_splits=5,
    C=0.5,
    max_iter=2000,
    show_eda=False,
    verbose=True,
    label_h=4,
    label_eps=0.001,
    feature_kwargs=feature_kwargs,
    tune_grid=(0.55, 0.60, 0.65),
    save_plots=True,                         # turn on artifact saving
    out_dir="simplest_ml/_exp",          # where to save
    run_id=None                             # auto timestamp

)
summarize_results(results)


Using data_dir = data

[BTC] ==== PIPELINE START (v3) ====
[BTC] LOAD: rows=17568, cols=11, span=2023-10-30 00:59:59.999000+00:00 → 2025-10-30 23:59:59.999000+00:00, cols_req_present=True
[BTC] FEATURES+LABEL: windows=(3, 6, 12, 24) H=4 eps=0.001
[BTC] prepare_Xy: input rows=17568 [2023-10-30 00:59:59.999000+00:00 → 2025-10-30 23:59:59.999000+00:00]
[BTC] add_features: start, rows=17568, windows=(3, 6, 12, 24)
[BTC] add_features: generated 67 features; total NaNs after lag=787
[BTC] make_label(3-class): H=4 eps=0.001 counts={1: 7852, -1: 7292, 0: 2424}
[BTC] prepare_Xy: dropped NaNs -> rows 17568 → 17518; features=67
[BTC] TRAIN: TSCV n_splits=5, C=0.5
[BTC] train: fold 1/5 | train=2923 test=2919
[BTC] train: fold 1 metrics | macroAUC=0.5464 ACC=0.4460
[BTC] train: fold 2/5 | train=5842 test=2919
[BTC] train: fold 2 metrics | macroAUC=0.5460 ACC=0.4669
[BTC] train: fold 3/5 | train=8761 test=2919
[BTC] train: fold 3 metrics | macroAUC=0.5726 ACC=0.4502
[BTC] train: fold 4/5 | train=116